# Machine Learning

Examples on performing ML in PySpark.

## Creating a Spark session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sandbox").getOrCreate()

## Reading Data

In [3]:
df = spark.read.csv('../data/test1.csv', header=True, inferSchema=True)

## Displaying Data

In [4]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



## Grouping Features

In Pyspark, features that will be used for training needs to be grouped with a `VectorAssembler` before sending it for training.

In [5]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

Transforming the dataframe

In [6]:
df = assembler.transform(df)
df.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [7]:
sanitized_df = df.select('Independent Features', 'Salary')

## Linear Regression

### Split Data to Train, Test

In [8]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = sanitized_df.randomSplit([0.75, 0.25])

### Training

In [10]:
regressor = LinearRegression(featuresCol="Independent Features", labelCol="Salary")
model = regressor.fit(train_data)

### Coefficients

In [11]:
model.coefficients

DenseVector([-79.3651, 1693.1217])

### Intercept

In [12]:
model.intercept

15714.285714285747

### Predictions

In [13]:
pred = model.evaluate(test_data)
pred.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|17275.132275132273|
|          [30.0,8.0]| 25000| 26878.30687830688|
+--------------------+------+------------------+



### MAE and MSE

In [14]:
pred.meanAbsoluteError, pred.meanSquaredError

(1301.587301587304, 2026734.9738249276)